Load data and compute the target stats

In [1]:
%load_ext autoreload
%autoreload 2

from target_stats import *
from subgroup_labeling import *

from mcbo.optimizers.non_bo.simulated_annealing import SimulatedAnnealing
from mcbo.optimizers.bo_builder import BoBuilder 
from mcbo.acq_funcs.factory import acq_factory

from tqdm import tqdm

data = pd.read_csv('df_full.csv')
all_std_stats = []
for g in [0, 1]:
    all_std_stats.extend([
        f"target_HR_{g}", f"target_HR_{g}_lcl", f"target_HR_{g}_ucl"
    ])
    # Median keys
    for z in [0, 1]:
        base = f"target_{g}_z{z}_median"
        all_std_stats.extend([
            base, f"{base}_lcl", f"{base}_ucl"
        ])

target_stats = get_est_stats(data, label=data['label'])
stats_dict = target_stats.to_dict(orient='records')[0]
stats_dict

Running on device: cpu


{'target_HR_0': 1.11,
 'target_HR_0_lcl': 0.8,
 'target_HR_0_ucl': 1.54,
 'target_0_z0_median': 8.9,
 'target_0_z0_median_lcl': 7.0,
 'target_0_z0_median_ucl': 12.0,
 'target_0_z1_median': 6.8,
 'target_0_z1_median_lcl': 3.4,
 'target_0_z1_median_ucl': 9.8,
 'target_HR_1': 0.59,
 'target_HR_1_lcl': 0.42,
 'target_HR_1_ucl': 0.84,
 'target_1_z0_median': 6.8,
 'target_1_z0_median_lcl': 5.2,
 'target_1_z0_median_ucl': 7.9,
 'target_1_z1_median': 10.4,
 'target_1_z1_median_lcl': 8.3,
 'target_1_z1_median_ucl': 14.7}

Define the task

In [2]:
df = data[['time', 'event', 'Z']]

N = 400
k = 200

task = SubgroupLabeling(n=len(df), k=k, data=df, target_stats=stats_dict)
search_space = task.get_search_space()
input_constraints = task.input_constraints

Example of defining a simple SA optimizer, which unsurprisingly failde because of the constraint

In [ ]:
opt = SimulatedAnnealing(
    search_space=search_space,
    input_constraints=input_constraints,
    obj_dims=[0],
    out_constr_dims=None,
    out_upper_constr_vals=None
    )

In [ ]:
%xmode Plain

budget_eval = 200

for _ in tqdm(range(budget_eval)):
    x_next = opt.suggest()
    y_next = task(x_next)
    
    y_fixed = np.array(y_next)
    if y_fixed.ndim == 0:
        y_fixed = y_fixed.reshape(1, 1)
    elif y_fixed.ndim == 1:
        y_fixed = y_fixed.reshape(-1, 1)

    opt.observe(x_next, y_fixed)
    
print(opt.best_x, opt.best_y)

Function to generate original pick

In [3]:
def generate_valid_design(N, k, n_samples):
    data = []
    for _ in range(n_samples):
        row = np.zeros(N); row[:k] = 1; np.random.shuffle(row)
        data.append(row)
    return pd.DataFrame(data, columns=[f'g{i+1}' for i in range(N)])

Define the Bayesian optimizer.

In [19]:
from johnson_acq_optimizer import JohnsonSAAcqOptimizer, JohnsonLSAcqOptimizer
from johnson_exact_gp import JohnsonExactGPModel    

# 1. Build Standard Shell using BoBuilder
print("Building optimizer shell...")
builder = BoBuilder(model_id='gp_rd', acq_opt_id='ls', acq_func_id='ei', tr_id='basic') # define as default setting, later rewrite
optimizer = builder.build_bo(search_space=search_space, n_init=10)
    
# 2. INJECT Custom Components (Hot-Swap)
print("Injecting Custom Johnson Components...")
    
# A. Swap the Model
custom_model = JohnsonExactGPModel(search_space, k_size=k, n_size=N, diffusion=True)
optimizer.model = custom_model
    
# B. Swap the Acquisition Optimizer -> USING SA NOW
print("Using Simulated Annealing Optimizer...")
custom_acq_opt1 = JohnsonSAAcqOptimizer(
    search_space, 
    n_swaps=30, 
    n_steps=2000, 
    T_max=1.0, 
    cooling=0.995,
    input_constraints=input_constraints,
    obj_dims=[0],
    out_constr_dims=None,
    out_upper_constr_vals=None
    )
custom_acq_opt2 = JohnsonLSAcqOptimizer(
    search_space, 
    n_swaps=10, 
    n_steps=1000,
    input_constraints=input_constraints,
    obj_dims=[0],
    out_constr_dims=None,
    out_upper_constr_vals=None
    )
optimizer.acq_optimizer = custom_acq_opt1
    
# C. Swap the Acquisition Function
optimizer.acq_func = acq_factory('ei', model=custom_model)

Building optimizer shell...
Injecting Custom Johnson Components...
Using Simulated Annealing Optimizer...


Starting Optimization Loop

In [ ]:
np.random.seed(66)

# 3. Prime with Valid Data
print("Generating Initial Data...")
x_init = generate_valid_design(N, k, 20)
y_init = task.evaluate(x_init)

y_fixed = np.array(y_init)
y_fixed = y_fixed.reshape(-1, 1)

optimizer.observe(x_init, y_fixed)

for i in range(50):
    x_next = optimizer.suggest(1)
    y_next = task.evaluate(x_next)

    y_fixed = np.array(y_next)
    if y_fixed.ndim == 0:
        y_fixed = y_fixed.reshape(1, 1)
    elif y_fixed.ndim == 1:
        y_fixed = y_fixed.reshape(-1, 1)

    optimizer.observe(x_next, y_fixed)

    best_y_val = optimizer.best_y
    if torch.is_tensor(best_y_val):
         best_y_val = best_y_val.item()
            
    print(f"Iter {i+1}: Best Y = {best_y_val:.4f}")


Generating Initial Data...
Iter 1: Best Y = 0.2167
Iter 2: Best Y = 0.2167
Iter 3: Best Y = 0.2167
Iter 4: Best Y = 0.2167
Iter 5: Best Y = 0.2167
Iter 6: Best Y = 0.2167
Iter 7: Best Y = 0.2167
Iter 8: Best Y = 0.2167
Iter 9: Best Y = 0.2167
Iter 10: Best Y = 0.2167
Iter 11: Best Y = 0.1899
Iter 12: Best Y = 0.1899
Iter 13: Best Y = 0.1899
Iter 14: Best Y = 0.1899
Iter 15: Best Y = 0.1899
Iter 16: Best Y = 0.1899
Iter 17: Best Y = 0.1899
Iter 18: Best Y = 0.1899
Iter 19: Best Y = 0.1899
Iter 20: Best Y = 0.1899
